1.Submit a Google Colab notebook containing your completed code and experimentation results.

2.Include comments and explanations in your code to help understand the implemented logic.

**Additional Notes:**
*   Ensure that the notebook runs successfully in Google Colab.
*   Document any issues encountered during experimentation and how you addressed them.

**Grading:**
*   Each task will be graded out of the specified points.
*   Points will be awarded for correctness, clarity of code, thorough experimentation, and insightful analysis.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

SOURCE_DIR = '/content/Q3_data.csv'

Mounted at /content/gdrive


In [2]:
import torch
import re
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
import math
from gensim.models import Word2Vec

In [3]:
def delete_hashtag_usernames(text):
  try:
    result = []
    for word in text.split():
      if word[0] not in ['@', '#']:
        result.append(word)
    return ' '.join(result)
  except:
    return ''

def delete_url(text):
  text = re.sub(r'http\S+', '', text)
  return text

def delete_ex(text):
  text = re.sub(r'\u200c', '', text)
  return text

# 0. Data preprocessing

In [4]:
!pip install json-lines

In [5]:
import json_lines

In [8]:
# 1. extract all tweets from file and save them in memory
# 2. remove urls, hashtags and usernames. use the prepared functions

# with open(SOURCE_DIR, 'r', encoding='utf-8') as file:
#     # Iterating over each JSON object in the file using json_lines.reader
#     for tweet in json_lines.reader(file):
#         tweet_text = tweet['Text']
#         # Applying preprocessing functions to remove hashtags, usernames, and URLs
#         tweet_text = delete_hashtag_usernames(tweet_text)
#         tweet_text = delete_url(tweet_text)
#         tweet_text = delete_ex(tweet_text)
#         tweets.append(tweet_text)

# Load the dataset
df = pd.read_csv(SOURCE_DIR, engine='python')

# Function to clean a single tweet with prepared functions
def clean_tweet(tweet):
    tweet = delete_hashtag_usernames(tweet)
    tweet = delete_url(tweet)
    tweet = delete_ex(tweet)
    return tweet

# Clean all tweets in the dataframe
df['cleaned_text'] = df['Text'].apply(clean_tweet)


In [9]:
# Showing first 10 rows:
df.head(10)

,Unnamed: 0,Datetime,Text,PureText,Language,Sentiment,Date,cleaned_text
0,0,2022-09-22 09:14:35+00:00,بنشین تا شود نقش فال ما \nنقش هم‌ فردا شدن\n#م...,بنشین تا شود نقش فال ما نقش هم‌ فردا شدن,fa,negative,2022-09-22,بنشین تا شود نقش فال ما نقش هم فردا شدن
1,1,2022-10-06 01:44:55+00:00,@Tanasoli_Return @dr_moosavi این گوزو رو کی گر...,این گوزو رو کی گردن میگیره؟؟ دچار زوال عقل شده...,fa,very negative,2022-10-06,این گوزو رو کی گردن میگیره؟؟ دچار زوال عقل شده...
2,2,2022-09-22 15:12:28+00:00,@ghazaleghaffary برای ایران، برای مهسا.\n#OpIr...,برای ایران، برای مهسا.,fa,positive,2022-09-22,برای ایران، برای مهسا.
3,3,2022-09-22 09:35:50+00:00,@_hidden_ocean مرگ بر دیکتاتور \n#OpIran \n#Ma...,مرگ بر دیکتاتور,fa,very negative,2022-09-22,مرگ بر دیکتاتور
4,4,2022-09-22 01:31:25+00:00,نذاریم خونشون پایمال شه.‌‌.‌‌.\n#Mahsa_Amini #...,نذاریم خونشون پایمال شه.‌‌.‌‌.,fa,negative,2022-09-22,نذاریم خونشون پایمال شه...
5,5,2022-09-26 21:05:14+00:00,@Nabauti88 مابهت افتخار میکنیم نبات باعث شدی ک...,مابهت افتخار میکنیم نبات باعث شدی کل دنیا مارو...,fa,positive,2022-09-26,مابهت افتخار میکنیم نبات باعث شدی کل دنیا مارو...
6,6,2022-09-22 20:37:50+00:00,@Bunnpaw برای انسانای خوشگلمون\n\n#مهسا_امینی ...,برای انسانای خوشگلمون,fa,positive,2022-09-22,برای انسانای خوشگلمون
7,7,2022-09-28 05:27:37+00:00,@neginsh فارغ از هر باوری متحد شویم.\n#مهسا_ام...,فارغ از هر باوری متحد شویم.,fa,no sentiment expressed,2022-09-28,فارغ از هر باوری متحد شویم.
8,8,2022-10-09 03:40:09+00:00,@mansurehhossai2 اینها عجب موجودات پستی هستن🥺🥺...,اینها عجب موجودات پستی هستن🥺🥺🥺الهی بگردم، من خ...,fa,very negative,2022-10-09,اینها عجب موجودات پستی هستن🥺🥺🥺الهی بگردم، من خ...
9,9,2022-09-24 21:46:20+00:00,@ShahinMaghsoodi کصخلا چرا ۴ تاوفحشش نمیدن؟\n\...,کصخلا چرا ۴ تاوفحشش نمیدن؟,fa,very negative,2022-09-24,کصخلا چرا ۴ تاوفحشش نمیدن؟


# 1. Functions

## Cosine Similarity

To measure the similarity between two words, you need a way to measure the degree of similarity between two embedding vectors for the two words. Given two vectors $u$ and $v$, cosine similarity is defined as follows:

$$\text{CosineSimilarity(u, v)} = \frac {u \cdot v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}$$

* $u \cdot v$ is the dot product (or inner product) of two vectors
* $||u||_2$ is the norm (or length) of the vector $u$
* $\theta$ is the angle between $u$ and $v$.
* The cosine similarity depends on the angle between $u$ and $v$.
    * If $u$ and $v$ are very similar, their cosine similarity will be close to 1.
    * If they are dissimilar, the cosine similarity will take a smaller value.

<img src="images/cosine_sim.png" style="width:800px;height:250px;">
<caption><center><font color='purple'><b>Figure 1</b>: The cosine of the angle between two vectors is a measure of their similarity.</font></center></caption>

Implement the function `cosine_similarity()` to evaluate the similarity between word vectors.

**Reminder**: The norm of $u$ is defined as $ ||u||_2 = \sqrt{\sum_{i=1}^{n} u_i^2}$

In [12]:
def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similarity between u and v

    Arguments:
        u -- a word vector of shape (n,)
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """

    # Compute the dot product of vectors u and v
    dot_product = np.dot(u, v)

    # Compute the norms of vectors u and v
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)

    # Compute the cosine similarity
    cosine_similarity = dot_product / (norm_u * norm_v)

    return cosine_similarity

## find k nearest neighbors

In [13]:
def find_k_nearest_neighbors(word, embedding_dict, k):
    """
    implement a function to return the nearest words to an specific word based on the given dictionary

    Arguments:
        word           -- a word, string
        embedding_dict -- dictionary that maps words to their corresponding vectors
        k              -- the number of word that should be returned

    Returns:
        a list of size k consisting of the k most similar words to the given word

    Note: use the cosine_similarity function that you have implemented to calculate the similarity between words
    """
    similarities = {}


    # Checking if the word is in the embedding dictionary
    if word not in embedding_dict:
        return []  # Return an empty list if the word is not found in the embedding dictionary

    # Getting the embedding vector of the given word
    word_vector = embedding_dict[word]

    # Calculate the cosine similarity between the given word vector and all other word vectors
    similarities = [(other_word, cosine_similarity(word_vector, other_vector))
                    for other_word, other_vector in embedding_dict.items() if other_word != word]

    # Sort the words based on their cosine similarity scores in descending order
    similarities.sort(key=lambda x: x[1], reverse=True)

    # Select the top k words (excluding the given word itself) as the k nearest neighbors
    nearest_neighbors = [word for word, _ in similarities[:k]]

    return nearest_neighbors

# 2. One hot encoding

In [14]:
# 1. find one hot encoding of each word
# One-hot encode each word
words = df['cleaned_text'].str.split().explode().unique()

# Reshaping words to be a 2D array
words = np.array(words).reshape(-1, 1)

# Initializing the OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Fitting and transforming the words with the encoder
onehot = encoder.fit_transform(words)

# Creating a dictionary that maps each word to its one-hot encoding
embedding_dict = {word: onehot[i] for i, word in enumerate(words.flatten())}

# 2. find 10 nearest words from "آزادی"
nearest_neighbors = find_k_nearest_neighbors("آزادی", embedding_dict, 10)
print(f"The 10 nearest neighbors to <آزادی> are: {nearest_neighbors}")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


The 10 nearest neighbors to <آزادی> are: ['بنشین', 'تا', 'شود', 'نقش', 'فال', 'ما', 'هم', 'فردا', 'شدن', 'این']


##### Describe advantages and disadvantages of one-hot encoding

Advantage:


1. Simple and Intuitive: One-hot encoding is straightforward to understand and implement. It assigns a unique binary representation to each category, making it easy to interpret.
2. Preserves Information: Each category is represented by a unique binary vector, preserving the original information about the categories without any assumptions or biases.
3. Compatibility: One-hot encoding is compatible with a wide range of machine learning algorithms and models. Many algorithms, including linear models, decision trees, and neural networks, can directly handle one-hot encoded features.
4. Insensitivity to Magnitude: One-hot encoding treats all categories as equally distant from each other in the feature space. It doesn't introduce any notion of magnitude or ordinal relationship between categories, which can be advantageous in certain applications.

Disadvantage:
1. High Dimensionality: One-hot encoding can lead to a high-dimensional feature space, especially when dealing with categorical variables with many unique categories. This can increase the computational complexity and memory requirements of the model.
2. Sparsity: In high-dimensional one-hot encoded representations, most of the elements in the vector are zero (except for one element representing the category). This sparsity can lead to inefficiency in terms of storage and computation, especially if the majority of categories are rare.
3. Loss of Information: One-hot encoding discards any intrinsic relationships or similarities between categories. It treats all categories as independent and unrelated, which may not always be the case in real-world data.
4. Curse of Dimensionality: The high-dimensional nature of one-hot encoded representations can lead to the curse of dimensionality, where the amount of data required to generalize accurately grows exponentially with the number of features.
5. Inefficient for Continuous Data: One-hot encoding is primarily designed for categorical data. It's not suitable for continuous variables, as it would require binning or discretization, which may lead to loss of information or introduce artificial boundaries.

# 3. TF-IDF

In [23]:
# 1. find the TF-IDF of all tweets.
# 2. choose one tweets randomly.
# 3. find 10 nearest tweets from chosen tweet.

import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# step 1: finding the TF-IDF of all tweets.
cleaned_tweets = df['cleaned_text']

# Initializing the TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fitting and transforming the tweets with the vectorizer
tfidf_matrix = vectorizer.fit_transform(cleaned_tweets)

# Step 2: Choosing one tweet randomly
random_tweet = random.choice(cleaned_tweets)

# Print the random tweet
print(random_tweet)

# Step 3: Finding 10 nearest tweets from the chosen tweet
# Calculate TF-IDF representation of the chosen tweet
chosen_tweet_tfidf = vectorizer.transform([random_tweet])

# Calculate cosine similarity between the chosen tweet and all other tweets
similarities = cosine_similarity(chosen_tweet_tfidf, tfidf_matrix)

# Get indices of 10 nearest tweets (excluding the chosen tweet itself)
nearest_tweet_indices = np.argsort(similarities[0])[-11:-1]

# Retrieve the nearest tweets
nearest_tweets = [cleaned_tweets[i] for i in nearest_tweet_indices]

# Print the nearest tweets
print("Nearest tweets:")
for i, tweet in enumerate(nearest_tweets, 1):
    print(f"{i}. {tweet}")

دست از هشتگ زدن برندارید
Nearest tweets:
1. به هشتگ زدن ادامه میدیم
2. به هشتگ زدن ادامه میدیم
3. این هم قدرت هشتگ زدن 
4. درحال هشتگ زدن...🥲😂
5. جهت. هشتگ زدن
6. جهت هشتگ زدن
7. از صبح تا حالا انقدر هشتگ زدم کع دیگه چشمام نمیبینه😭😭😭 لطفااااا دست برندارید از هشتگ زدن Help us!!we are loosing internet😭😭
8. فقط برای هشتگ زدن🥲
9. از هشتگ زدن دست برنداریددد
10. *برای هشتگ زدن


##### Describe advantages and disadvantages of TF-IDF

Advatages:
1. Weighting by Importance: TF-IDF assigns higher weights to terms that are more informative or discriminating within a document relative to the entire corpus. This allows it to capture the significance of terms in the context of individual documents and the corpus as a whole.
2. Simple yet Effective: TF-IDF is relatively simple to implement and computationally efficient, making it a practical choice for many text processing tasks.
3. Reduction of Noise: By downweighting terms that appear frequently across many documents (i.e., have high inverse document frequency), TF-IDF helps in reducing the impact of noisy or common terms, such as stopwords, in the document representation.
4. Flexibility: TF-IDF can be adapted to suit various text processing scenarios by adjusting parameters such as the weighting scheme, normalization method, and handling of special cases (e.g., document length normalization).
5. Interpretability: The resulting TF-IDF matrix provides interpretable representations of documents, where each dimension represents a term and the value reflects its importance in the document.

Disadvantages:
1. Bag-of-Words Assumption: TF-IDF relies on the bag-of-words assumption, treating documents as collections of independent terms without considering the order or context in which they appear. This may lead to loss of important semantic information, especially in tasks requiring understanding of word sequences or phrases.
2. Vocabulary Size: The size of the vocabulary (i.e., the number of unique terms) can grow large, especially in large document collections, leading to high-dimensional feature spaces and potentially increased computational complexity.
3. Sparse Representations: TF-IDF representations often result in sparse matrices, where most entries are zero, since most terms do not appear in most documents. Dealing with sparse data can pose challenges in storage, computation, and memory usage, especially in large-scale applications.
4. Sensitivity to Preprocessing: The performance of TF-IDF can be sensitive to preprocessing steps such as tokenization, stemming, and stopword removal. Small changes in preprocessing choices may lead to significant differences in the resulting document representations.
5. Lack of Semantic Understanding: TF-IDF does not explicitly capture semantic relationships between terms or documents. It treats terms as independent entities based solely on their statistical properties within the corpus, without considering their meanings or contexts.

# 4. Word2Vec

In [24]:
# 1. train a word2vec model base on all tweets
# 2. find 10 nearest words from "آزادی"

# Step 1: Training a Word2Vec model based on all tweets
tweets = df['cleaned_text'].str.split().tolist()

# Initialize the Word2Vec model
word2vec_model = Word2Vec(sentences=tweets, vector_size=100, window=5, min_count=1, workers=4)

# Step 2: Finding 10 nearest words from "آزادی" using the trained Word2Vec model
nearest_words = word2vec_model.wv.most_similar("آزادی", topn=10)

# Print the nearest words
print("10 Nearest words from 'آزادی':")
for word, similarity in nearest_words:
    print(f"{word}: {similarity}")

10 Nearest words from 'آزادی':
ازادی: 0.9977310299873352
آزادی.: 0.9953532814979553
آزادی،: 0.9942513108253479
زندگی،: 0.9932334423065186
زن،: 0.9908443689346313
وطنم: 0.9907423853874207
زندگی: 0.9876235127449036
زن: 0.9875372052192688
ایرانمون: 0.9873859882354736
ایرانم: 0.9866814613342285


##### Describe advantages and disadvantages of Word2Vec

Advantages:
1. Semantic Similarity: Word2Vec captures semantic similarities between words by representing them as vectors in a continuous vector space. Words with similar meanings tend to have similar vector representations, allowing for more meaningful comparisons and analyses.
2. Dimensionality Reduction: Word2Vec reduces the high-dimensional nature of text data by mapping words into a lower-dimensional space, typically a few hundred dimensions. This makes it computationally efficient and practical for various natural language processing tasks.
3. Compositionality: Word2Vec embeddings support compositionality, meaning that the vector representations of phrases or sentences can be obtained by combining the embeddings of constituent words. This property enables the model to capture syntactic and semantic relationships at different levels of granularity.
4. Generalization: Word2Vec embeddings generalize well to unseen words or contexts that were not present in the training data. This is because the model learns distributed representations based on the co-occurrence patterns of words, allowing it to capture underlying linguistic regularities.
5. Efficiency: Word2Vec training can be efficiently parallelized and scaled to large datasets using techniques like stochastic gradient descent and negative sampling. This makes it suitable for training on large-scale text corpora, including web-scale datasets.

Disadvantages:
1. Lack of Interpretability: While Word2Vec embeddings capture semantic similarities, the individual dimensions of the embedding space do not have clear semantic interpretations. Understanding the meaning of each dimension is challenging, limiting the interpretability of the learned embeddings.
2. Contextual Ambiguity: Word2Vec treats each word as a single point in the embedding space, ignoring the variability in word meanings across different contexts. As a result, words with multiple senses or ambiguous meanings may have blurred representations, leading to ambiguity in certain cases.
3. Out-of-Vocabulary Words: Word2Vec may struggle with out-of-vocabulary words that are not present in the training corpus or have rare occurrences. Such words may not have meaningful embeddings, potentially impacting the performance of downstream tasks, especially in specialized domains.
4. Fixed-Length Context Window: Word2Vec uses a fixed-length context window to capture word co-occurrences, which may not capture long-range dependencies or contextual information beyond the window size. This limitation can affect the quality of embeddings for words that depend on broader contexts.
5. Data Dependency: The quality of Word2Vec embeddings heavily depends on the quality, size, and diversity of the training data. Biases, noise, or imbalance in the training corpus can propagate to the learned embeddings, potentially leading to biased or suboptimal representations.

# 5. Contextualized embedding

In [25]:
!pip install transformers[sentencepiece]

In [29]:
# Load model and tokenizer

from transformers import BertModel, BertTokenizer, BertForSequenceClassification

model_name = "HooshvareLab/bert-base-parsbert-uncased"

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=6)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
import torch.nn as nn
from tqdm import tqdm
from transformers import BertModel, BertTokenizer, AdamW, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch

# 1. fine-tune the model base on all tweets
# 2. find 10 nearest words from "آزادی"

# Step 1: Fine-tuning the model
class TweetDataset(Dataset):
    def __init__(self, df, tokenizer):
        # Converting the 'cleaned_text' column to a list
        self.tweets = df['cleaned_text'].tolist()

        # Converting the 'Sentiment' column to a list
        self.labels = df['Sentiment'].map({'negative': 0, 'very negative': 1, 'positive': 2, 'no sentiment expressed': 3, 'very positive': 4, 'mixed': 5}).tolist()

        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.tweets)

    def __getitem__(self, idx):
        tweet = self.tweets[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            tweet,
            truncation=True,
            padding='max_length',
            max_length=128,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


dataset = TweetDataset(df, tokenizer)
dataloader = DataLoader(dataset, batch_size=8)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)
model.train()

for batch in tqdm(dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)

    labels = batch['labels']

    # Defining the loss function
    loss_function = nn.CrossEntropyLoss()

    # Forward passing through the model
    outputs = model(**batch)

    # Computing the loss
    loss = loss_function(outputs.logits, labels)

    print(f"==================== The loss is: {loss} ====================")

    # Backward passing and optimization
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/2500 [00:00<?, ?it/s]

==================== The loss is: 1.8657903671264648 ====================


  0%|          | 2/2500 [00:01<31:04,  1.34it/s]  

==================== The loss is: 1.8475978374481201 ====================


  0%|          | 3/2500 [00:01<21:20,  1.95it/s]

...

==================== The loss is: 0.6964752078056335 ====================


100%|██████████| 2500/2500 [10:33<00:00,  3.95it/s]

==================== The loss is: 0.7427964210510254 ====================



  0%|          | 0/100000 [00:00<?, ?it/s]


ValueError: Expected 2D array, got 1D array instead:
array=[-0.1408102 -1.8788577  0.6819205  4.43898   -1.9560708 -2.0394385].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [33]:
# Step 2: Finding the 10 nearest words from "آزادی" based on the contextualized embeddings
from scipy.spatial.distance import cosine

input_ids = tokenizer.encode('آزادی', return_tensors='pt').to(device)
with torch.no_grad():
    azadi_embedding = model(input_ids)[0][0, :].cpu().numpy()

# Calculate the cosine similarity with all other words
similarities = []
for word in tqdm(tokenizer.get_vocab()):
    input_ids = tokenizer.encode(word, return_tensors='pt').to(device)
    with torch.no_grad():
        word_embedding = model(input_ids)[0][0, :].cpu().numpy()
    similarity = 1 - cosine(azadi_embedding, word_embedding)
    similarities.append((word, similarity))

# Sort by similarity and get the top 10 words
similar_words = sorted(similarities, key=lambda x: x[1], reverse=True)[:10]
print("similar words to 'آزادی' are:", similar_words)

100%|██████████| 100000/100000 [18:22<00:00, 90.71it/s]

similar words to 'آزادی' are: [('سرگروههای', 0.9996515512466431), ('بزرگراهی', 0.9996299147605896), ('یکپارچهسازی', 0.9996269941329956), ('محققان', 0.9994257092475891), ('حلیم', 0.9993884563446045), ('ورزشگاههای', 0.9992910027503967), ('جهاد', 0.9992689490318298), ('express', 0.9992390871047974), ('۳۳۶', 0.9991313815116882), ('مستند', 0.9991137385368347)]


##### Describe advantages and disadvantages of Contextualized embedding

Advantages:
1. Context Sensitivity: Contextualized embeddings capture the meaning of a word based on its surrounding context within a sentence or document. This enables them to handle polysemy (words with multiple meanings) and capture nuances in meaning that are dependent on context.
2. Semantic Representations: Contextualized embeddings provide rich semantic representations that reflect the meaning of words within their respective contexts. This enables them to capture complex linguistic relationships and semantic similarities between words.
3. Transfer Learning: Pre-trained contextualized embedding models, such as BERT, can be fine-tuned on specific downstream tasks with minimal task-specific data. This transfer learning approach allows leveraging the knowledge encoded in the pre-trained model for a wide range of natural language processing tasks, leading to improved performance.
4. Out-of-Vocabulary Handling: Contextualized embeddings can handle out-of-vocabulary words by leveraging subword tokenization and contextual information from surrounding words. This enables them to generate meaningful embeddings for words not seen during training.
5. Robustness to Syntax and Grammar: Contextualized embeddings capture syntactic and grammatical structures in addition to semantic meaning. This enables them to generate embeddings that reflect the grammatical correctness and syntactic structure of sentences.

Disadvantages:
1. Computationally Expensive: Generating contextualized embeddings using models like BERT can be computationally expensive, especially for large documents or datasets. This limits their scalability and may require significant computational resources for training and inference.
2. Lack of Transparency: Contextualized embeddings are complex, high-dimensional vectors that lack transparency in terms of their individual dimensions' semantic interpretations. Understanding the specific meaning encoded in each dimension of the embedding space is challenging.
3. Data Dependency: Contextualized embeddings require large amounts of annotated text data for pre-training and fine-tuning. The quality and diversity of the training data significantly affect the performance of the resulting embeddings, potentially leading to biased or suboptimal representations.
4. Interpretability: While contextualized embeddings capture rich semantic information, interpreting their embeddings to understand why specific decisions were made by downstream models can be challenging. This lack of interpretability may hinder the model's trustworthiness and usability in certain applications.
5. Fixed-Length Context: Contextualized embeddings have a fixed context window size determined by the architecture of the pre-trained model. This fixed-length context may not capture long-range dependencies or contextual information beyond the window size, limiting the model's ability to capture certain linguistic phenomena.